In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import pickle
import torch.nn.functional as F
from collections import Counter
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
from pytorch_pretrained_bert import BertForSequenceClassification,BertConfig
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
# MODEL = 'bert-base-multilingual-uncased'

ratings = pd.read_excel("ratings.xlsx", encoding='utf-8', sep='\t', header=[0])
data = pd.read_excel("10k moviemetadatahindi.xlsx", encoding='utf-8', sep='\t', header=[0],sheet_name='Sheet2')
#data.head()
print(ratings.head())
print(data.head())

   userId  movieId  rating   timestamp  mov_avg_rating
0       1      110     1.0  1425941529        4.025251
1       1      147     4.5  1425942435        3.462500
2       1      858     5.0  1425941523        4.333763
3       1     1221     5.0  1425941546        4.274766
4       1     1246     5.0  1425941556        3.911153
   adult  belongs_to_collection    budget  \
0  False                    NaN  30000000   
1  False                    NaN  65000000   
2  False                    NaN         0   
3  False                    NaN  16000000   
4  False                    NaN         0   

                                              genres  \
0  [{'आ ी डी': 16, 'नाम': 'एनीमेशन'}, {'आ ी डी': ...   
1  [{'आ ी डी': 12, 'नाम': 'एडवेंचर'}, {'आ ी डी': ...   
2  [{'आ ी डी': 10749, 'नाम': 'रोमांस'}, {'आ ी डी'...   
3  [{'आ ी डी': 35, 'नाम': 'कॉमेडी'}, {'आ ी डी': 1...   
4                  [{'आ ी डी': 35, 'नाम': 'कॉमेडी'}]   

                                            genres.1     id   

In [19]:
dataset =  pd.merge(data[['id','imdb_id','overview','genres']].copy(),ratings, how = 'inner', left_on ='id', right_on = 'movieId')
#dataset = dataset[:50000].copy()
#print(len(dataset))
#Counter(dataset['rating'])
dataset.head()

,id,imdb_id,overview,genres,userId,movieId,rating,timestamp,mov_avg_rating
0,862,tt0114709,"वुडी के नेतृत्व में, एंडी के खिलौने उनके कमरे ...","[{'आ ी डी': 16, 'नाम': 'एनीमेशन'}, {'आ ी डी': ...",1923,862,3.0,858335006,3.454545
1,862,tt0114709,"वुडी के नेतृत्व में, एंडी के खिलौने उनके कमरे ...","[{'आ ी डी': 16, 'नाम': 'एनीमेशन'}, {'आ ी डी': ...",2103,862,5.0,946044912,3.454545
2,862,tt0114709,"वुडी के नेतृत्व में, एंडी के खिलौने उनके कमरे ...","[{'आ ी डी': 16, 'नाम': 'एनीमेशन'}, {'आ ी डी': ...",5380,862,1.0,878941641,3.454545
3,862,tt0114709,"वुडी के नेतृत्व में, एंडी के खिलौने उनके कमरे ...","[{'आ ी डी': 16, 'नाम': 'एनीमेशन'}, {'आ ी डी': ...",6177,862,4.0,859415226,3.454545
4,862,tt0114709,"वुडी के नेतृत्व में, एंडी के खिलौने उनके कमरे ...","[{'आ ी डी': 16, 'नाम': 'एनीमेशन'}, {'आ ी डी': ...",6525,862,4.0,857388995,3.454545


In [20]:
###Preprocessing of moviedata
stoi = {}
itos = {}
text_corpus = []
dictionary = []
indexed_number = []
tokenized_word = []
segment_masks = []
labels = dataset['rating']

minx, maxx = 10000, 0
batchSize = 64
totalSize =  len(dataset) 
dataSize= len(dataset)
# dataSize = int(totalSize / batchSize) * batchSize
# print(dataSize)

for i in range(0, 1):                                                     ## Replacing full stop with nothing ##
    dataset['overview'] = dataset['overview']+dataset['genres']
    dataset['overview'] = dataset['overview'].str.replace('।', '') 
    dataset['overview'] = dataset['overview'].str.replace(',', '') 
    dataset['overview'] = dataset['overview'].str.replace("'", '') 
    dataset['overview'] = dataset['overview'].str.replace(":", '') 
    dataset['overview'] = dataset['overview'].str.replace("{", '')
    dataset['overview'] = dataset['overview'].str.replace("}", '') 

#for i in range(0, dataSize): 
##         dataset['overview'].values[i] = dataset['overview'].values[i] + (dataset['genres'].values[i])
##         dataset['overview'].values[i] = "[CLS] "+ str(dataset['overview'].values[i])+ str(dataset['genres'].values[i]) +" [SEP]" 
#        dataset['overview'].values[i] = "[CLS] "+ str(dataset['overview'].values[i]) +" [SEP]"
##         dataset['overview'].values[i] = tokenizer.tokenize(dataset['overview'].values[i])
#        dataset['overview'].values[i] = (dataset['overview'].values[i]).split(" ")
## 
#        tokenized_text = dataset['overview'].values[i]
##         print(tokenized_text)
##         stoi = {e:i for i,e in enumerate(tokenized_text)}
##         indexed_tokens = stoi(tokenized_text)
##         print(indexed_tokens)
##         for a in zip(tokenized_text, indexed_tokens):
##             stoi.update({a[0]: a[1]})
##             itos.update({a[1]: a[0]})
#            
##         text_corpus.append([stoi[tok] for tok in tokenized_text]) 
##         print(text_corpus)
#        tokenized_word=tokenized_word+tokenized_text
#    
#        if  i % 10000 == 0 :
#            print(i)
##         tokenized_word.append(tokenized_text)
##         all_space_tokens=all_space_tokens.append(all_space_tokens)
## print(tokenized_word)
## print(len(stoi))    

In [21]:
#dataset['overview'] = dataset[['overview',].values[i] + (dataset['genres'].values[i])
dataset['overview'] =  dataset['overview'].apply(lambda k : list(k.split(" ")))
tokenized_text = dataset['overview'].tolist()

In [22]:
dataset['overview'].apply(lambda m : tokenized_word.append(m))

0         None
1         None
2         None
3         None
4         None
5         None
6         None
7         None
8         None
9         None
10        None
11        None
12        None
13        None
14        None
15        None
16        None
17        None
18        None
19        None
20        None
21        None
22        None
23        None
24        None
25        None
26        None
27        None
28        None
29        None
          ... 
303073    None
303074    None
303075    None
303076    None
303077    None
303078    None
303079    None
303080    None
303081    None
303082    None
303083    None
303084    None
303085    None
303086    None
303087    None
303088    None
303089    None
303090    None
303091    None
303092    None
303093    None
303094    None
303095    None
303096    None
303097    None
303098    None
303099    None
303100    None
303101    None
303102    None
Name: overview, Length: 303103, dtype: object

In [7]:
#import pickle
# pickle.dump(tokenized_word, "tokenized_word.bin")

#with open("binary_list.bin","wb") as f:
#    pickle.dump(tokenized_word, f)
#    
with open("binary_list.bin","rb") as f:
    tokenized_word = pickle.load(f)    

In [15]:
#from sklearn.feature_extraction.text import TfidfVectorizer
#from sklearn.feature_extraction.text import CountVectorizer
#
#dataset["overview"]=["".join(overview) for overview in dataset["overview"].values]
##print(dataset["overview"])
#
#vectorizer=TfidfVectorizer()
#response= vectorizer.fit_transform(dataset["overview"])
## print(response)
#list(vectorizer.vocabulary_.keys())[:100]
## print(vectorizer.get_feature_names())
print(data['overview'][4])

बस जब जॉर्ज बैंकों अपनी बेटी की शादी से बरामद किया गया है, वह खबर यह है कि वह गर्भवती है प्राप्त करता है ... और जॉर्ज की पत्नी नीना भी उम्मीद कर रही है कि। उन्होंने कहा कि उनके घर की बिक्री पर योजना बना रहा था, लेकिन यह एक योजना है कि है - जॉर्ज की तरह - दोनों एक पोता और अपनी खुद की एक बच्चा के आगमन के साथ बदलने के लिए होगा।


In [9]:
# print((tokenized_word))
my_list=[]
train_stoi = {e:i for i,e in enumerate(tokenized_word[:260000])}

my_list=[]
test_stoi = {e:i for i,e in enumerate(tokenized_word[260000:])}

#print(stoi)

# def CountFrequency(my_list): 
      
#     # Creating an empty dictionary  
#     freq = {} 
#     for items in my_list: 
#         freq[items] = my_list.count(items) 
      
#     for key, value in freq.items(): 
#         print ("% d : % d"%(key, value))
# print(CountFrequency(tokenized_word))        
        

# print(stoi)
# print(len(stoi))
#k = Counter(tokenized_word) 
#print(k.most_common(102))
# Counter(stoi.most_common(10))

In [24]:
print(tokenized_word[20])
stoi = {e:i for elist in tokenized_word  for i,e in enumerate(elist)}

['जुनूनी', 'मास्टर', 'चोर', 'नील', 'मक्सौली', 'लॉस', 'एंजिल्स', 'भर', 'में', 'विभिन्न', 'पागल', 'चोरियों', 'पर', 'एक', 'शीर्ष', 'पायदान', 'चालक', 'दल', 'की', 'ओर', 'जाता', 'है', 'जबकि', 'एक', 'मानसिक', 'रूप', 'से', 'अस्थिर', 'जासूस', 'विंसेंट', 'हैना', 'आराम', 'के', 'बिना', 'उसे', 'कर्मों', 'प्रत्येक', 'मनुष्य', 'को', 'पहचानता', 'है', 'और', 'क्षमता', 'और', 'अन्य', 'के', 'समर्पण', 'भले', 'ही', 'वे', 'जागरूक', 'अपनी', 'बिल्ली', 'और', 'चूहे', 'खेल', 'हिंसा', 'में', 'खत्म', 'हो', 'सकता', 'है', 'कर', 'रहे', 'हैं', 'सम्मान', 'करता', 'है[आ', 'ी', 'डी', '28', 'नाम', 'एक्शन', 'आ', 'ी', 'डी', '80', 'नाम', 'क्राइम', 'आ', 'ी', 'डी', '18', 'नाम', 'ड्रामा', 'आ', 'ी', 'डी', '53', 'नाम', 'थ्रिलर]']


In [25]:
for i in range(0, dataSize): 
        text_corpus.append([stoi[tok] for tok in tokenized_text[i]]) 
#         print(text_corpus)
        tokenized_word.append(tokenized_text)
    

for i in range(0, dataSize):
    if len(dataset['overview'].values[i]) < minx: minx = len(dataset['overview'].values[i])
    if len(dataset['overview'].values[i]) > maxx: maxx = len(dataset['overview'].values[i])

print("Max count :",maxx)
print("Min Count :",minx)        
        
def pad_sequence(corpus, max_length):                                                ## Adding padding
    feature = np.zeros((len(corpus), max_length), dtype=int)
    for i, row in enumerate(corpus):
        feature[i][-len(row):] = np.array(row)[:max_length]
    return feature

corpus = pad_sequence(text_corpus, maxx)
print(corpus)
print(len(corpus))


y_map = {e:i for i,e in enumerate(set(data['vote_average']))}
print(len(y_map))

Max count : 229
Min Count : 1
[[ 0  0  0 ... 59 50 33]
 [ 0  0  0 ... 59 50 33]
 [ 0  0  0 ... 59 50 33]
 ...
 [ 0  0  0 ... 43 50 27]
 [ 0  0  0 ... 43 50 27]
 [ 0  0  0 ... 50 51 52]]
303103
81


In [26]:
train, test = train_test_split(dataset, test_size=0.1)
print(len(train),len(test))

272792 30311


Max count : 229
Min Count : 1
[[ 0  0  0 ... 59 50 33]
 [ 0  0  0 ... 59 50 33]
 [ 0  0  0 ... 59 50 33]
 ...
 [ 0  0  0 ... 43 50 27]
 [ 0  0  0 ... 43 50 27]
 [ 0  0  0 ... 50 51 52]]
303103
81


In [27]:
def make_weights_for_balanced_classes(images, nclasses):                        
    count = [0] * nclasses                                                      
    for item in images:                                                         
        count[int(item)] += 1      
        
    weight_per_class = [0.] * nclasses                                      
    N = float(sum(count))                                                   
    
    for i in range(nclasses):                                                   
        weight_per_class[i] = N/float(count[i])                                 
    
    weight = [0] * len(images)                                              
    
    for idx, val in enumerate(images):                                          
        weight[idx] = weight_per_class[int(val)]                                  
    
    return weight  

In [34]:
##TRAIN
batchSize = 64
embed_size=100

totalSize =  len(train)
dataSize = int(totalSize / batchSize) * batchSize
# print(dataSize)

train_dataset=dataset[0:dataSize]
n_user=len(train_dataset['userId'].unique())
train_input=train_dataset['userId']
out_rating = (train_dataset['rating'][0:len(train_input)])
out_rating=out_rating*2
out_rating[0]=0
# print(output2)
output_size = len(out_rating.unique())
train_corpus=corpus[0:len(train_input)]
print(len(train_input))
print(len(train_corpus))
print(len(out_rating))
print(train_input)

##TEST

test_size = len(test)
test_size = int(test_size/batchSize) * batchSize
# print(test_size)
# print((dataSize2))
# print(len(corpus))
test_data=dataset[dataSize:dataSize+test_size]
test_input = test_data['userId']
test_output = test_data['rating']
test_output=test_output*2
# test_output[0]=0
test_corpus=corpus[dataSize:dataSize+test_size]
print(len(test_input))
print(len(test_output))
print(len(test_corpus))

weights =  make_weights_for_balanced_classes(out_rating.values,11)
weights = torch.DoubleTensor(weights)                                       
sampler = torch.utils.data.sampler.WeightedRandomSampler(weights, len(weights))                     
                                                                                
#train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=args.batch_size, shuffle = True,                              
#                                                             sampler = sampler, num_workers=args.workers, pin_memory=True)     



dset = TensorDataset(torch.from_numpy(np.array(train_input)),torch.from_numpy(np.array(train_corpus)), torch.from_numpy(np.array(out_rating)))

train_loader = DataLoader(dset,sampler=sampler, batch_size=batchSize)

tset= TensorDataset(torch.from_numpy(np.array(test_input)),torch.from_numpy(np.array(test_corpus)), torch.from_numpy(np.array(test_output)))
test_loader = DataLoader(tset,  shuffle = False, batch_size=batchSize)
# print(train_loader)

272768
272768
272768
0          1923
1          2103
2          5380
3          6177
4          6525
5          7050
6          7238
7          8659
8          9328
9          9682
10        10425
11         7016
12         7144
13         8659
14         9547
15          174
16          346
17          362
18          384
19          463
20          523
21          557
22          609
23          624
24          773
25          825
26          959
27         1107
28         1149
29         1608
          ...  
272738    10071
272739    10082
272740    10083
272741    10111
272742    10114
272743    10128
272744    10132
272745    10141
272746    10150
272747    10165
272748    10168
272749    10169
272750    10191
272751    10215
272752    10223
272753    10250
272754    10253
272755    10259
272756    10268
272757    10296
272758    10312
272759    10322
272760    10329
272761    10335
272762    10336
272763    10339
272764    10346
272765    10357
272766    10370
272767    10389
Nam

In [30]:
Counter(dataset['rating'])

Counter({3.0: 71039,
         5.0: 49355,
         1.0: 10472,
         4.0: 83197,
         4.5: 19997,
         2.0: 21673,
         3.5: 28633,
         2.5: 11834,
         0.5: 3394,
         1.5: 3509})

In [31]:
### Variables initialization
vocab_size = len(stoi)
print(len(stoi))
embed_size1 = 100
n_hidden = 128
n_layers = 3
seq_length = maxx
print(seq_length)

13750
229


In [32]:
print(n_user)
print(len(train_input))
# print(output.unique())
print((out_rating.unique()))
print(output_size)
print(max(dataset['userId']))

10231
272768
[ 0. 10.  2.  8.  6.  9.  4.  7.  5.  1.  3.]
11
10656


In [33]:
class UserEmbedding(nn.Module):
    def __init__(self,vocab_size, n_user,embed_size, n_hidden,n_layers,output_size, batchSize
                 ,seq_length, lstm_dropout=0.2 , batch_first=True,):
        
        super(UserEmbedding,self).__init__()
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.n_batch = batchSize
        
        self.movieembedding=nn.Embedding(24523000,embed_size)
        self.lstm_net =  nn.LSTM(embed_size,n_hidden,n_layers,dropout =0.5 , batch_first = batch_first)
        self.linear1 = nn.Linear(seq_length  * n_hidden , 256)
        self.linear2 = nn.Linear(256, 128)
        self.linear3 = nn.Linear(128, 64)
               
        self.embedding = nn.Embedding(24523000, embed_size)
        self.linear4 = nn.Linear(embed_size , 256)
        self.linear5 = nn.Linear(256, 128)
        self.linear6 = nn.Linear(128, 64)

        self.linear7 = nn.Linear(64+64,32) 
        self.linear8 = nn.Linear(32,output_size)
#         self.linear9 = nn.Linear(16, output_size)
               
    def forward(self, sents, users, hidden):
        sentEmbed = self.embedding(sents)
        lstm_out, hidden =  self.lstm_net(sentEmbed,hidden)
        lstm_out =  lstm_out.contiguous().view(self.n_batch,-1)  ## Changing size, reshape
        out1 = F.relu(self.linear1(lstm_out))
        out1 = F.relu(self.linear2(out1))
        out1 = F.relu(self.linear3(out1))
        
        userEmbed = self.embedding(users)
        out2 = F.relu(self.linear4(userEmbed))
        out2 = F.relu(self.linear5(out2))
        out2 = F.relu(self.linear6(out2))
        
        out3 = torch.cat((out1, out2), dim=1)
        out3 = F.relu(self.linear7(out3))
        out3 = self.linear8(out3)
#         out3 = self.linear9(out3)
              
        return out3,hidden

    def init_hidden(self):
        
        weight = next(self.parameters()).data            
        hidden = (weight.new(self.n_layers,self.n_batch,self.n_hidden).zero_(),
                  weight.new(self.n_layers,self.n_batch,self.n_hidden).zero_())
        
        return hidden 

In [19]:
model =  UserEmbedding(vocab_size,n_user,embed_size,n_hidden,n_layers,output_size,batchSize, seq_length,output_size)
print(model)
model.cuda()

In [ ]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [ ]:
model.train()
losses = []
total_loss=[]
predicted_rating=[]
accuracy_list=[]
for i in range(0,40):
    correct1=0
    
    for users,sentence, ratings in train_loader:
        model.zero_grad()
    
        hidden =  model.init_hidden()
        hidden = ([each.data for each in hidden])
        output1,hidden  = model(sentence,users,hidden)
        _, predicted= torch.topk(output1,k=1)  
        predicted=predicted.type(torch.DoubleTensor)
        correct=predicted.data.eq(ratings.data.view_as(predicted.data)).sum()    ###taking sum of all corrected values for a batch

        correct1=correct1+correct.item()
        lloss1= loss(output1, ratings.long())
        lloss1.backward()
        optimizer.step()
        
    accuracy=(correct1/len(train_input))*100
    accuracy_list.append(accuracy)    
    print(correct1/len(train_input),accuracy_list)  

In [ ]:
model.eval()
losses = []
total_loss=[]
predicted_rating=[]
accuracy_list=[]
for i in range(0,1):
    correct1=0
    
    for users,sentence, ratings in test_loader:
        model.zero_grad()
        hidden =  model.init_hidden()
        hidden = ([each.data for each in hidden])
        output1,hidden  = model(sentence,users,hidden)
        _, predicted= torch.topk(output1,k=1)  #returns topk values, indexes
        predicted=predicted.type(torch.DoubleTensor)
        correct=predicted.data.eq(ratings.data.view_as(predicted.data)).sum()    ###taking sum of all corrected values for a batch

        correct1=correct1+correct.item()
        lloss1= loss(output1, ratings.long())

    accuracy=(correct1/len(test_input))*100
    accuracy_list.append(accuracy)    
    print(correct1/len(test_input),accuracy_list)